In [1]:
import tensorflow as tf
import numpy as np
import os
import glob
import cv2
import pandas as pd
import time

In [2]:
TRAIN_DIR = './resources/images/gender/trainSet/'
Labels = os.listdir(TRAIN_DIR)
csvList = []

for dir in Labels:
    img_paths = glob.glob(TRAIN_DIR + dir + '/*')
    imgs = [cv2.imread(file, 0) for file in img_paths]
    
    for img in imgs:
        img = np.array(img, dtype=np.float32)
        img_flat = img.flatten()
        csvList.append(np.append(img_flat, int(dir.split('_')[0])))
        
        # image flip
        vertical_flip_img = cv2.flip(img, 1)
        vertical_flip_img = np.array(vertical_flip_img, dtype=np.float32)
        vertical_flip_img_flat = vertical_flip_img.flatten()
        csvList.append(np.append(vertical_flip_img_flat, int(dir.split('_')[0])))

In [3]:
csv_path = './resources/csv/gender/train_gender_img_label_data.csv'

if not os.path.isfile(csv_path):
    # dataFrame 생성
    df = pd.DataFrame(csvList)
    # 0번 col을 sorting -> 0번 row 부터 차례대로 읽어도 label(16384)은 sorting 안돼있음
    df = df.sort_values(0)
    # csv파일로 저장, (index 및 header 제외)
    df.to_csv(csv_path, sep=',', header=False, index=False)
else:
    print('file already exists...')

file already exists...


In [4]:
data = np.loadtxt('./resources/csv/gender/train_gender_img_label_data.csv', delimiter=',', dtype=np.float32)

train_img_data = data[:, 0:-1]
train_label_data = data[:, [-1]]

# min-max scaling
train_img_data = (train_img_data - train_img_data.min()) / (train_img_data.max() - train_img_data.min())
# contrast normalization
train_img_data = (train_img_data - train_img_data.mean()) / train_img_data.std()

In [5]:
TEST_DIR = './resources/images/gender/testSet/'
Labels = os.listdir(TEST_DIR)
csvList = []

for dir in Labels:
    img_paths = glob.glob(TEST_DIR + dir + '/*')
    imgs = [cv2.imread(file, 0) for file in img_paths]
    
    for img in imgs:
        img = np.array(img, dtype=np.float32)
        img_flat = img.flatten()
        csvList.append(np.append(img_flat, int(dir.split('_')[0])))

In [6]:
csv_path = './resources/csv/gender/test_gender_img_label_data.csv'

if not os.path.isfile(csv_path):
    # dataFrame 생성
    df = pd.DataFrame(csvList)
    # 0번 col을 sorting -> 0번 row 부터 차례대로 읽어도 label(16384)은 sorting 안돼있음
    df = df.sort_values(0)
    # csv파일로 저장, (index 및 header 제외)
    df.to_csv(csv_path, sep=',', header=False, index=False)
else:
    print('file already exists...')

file already exists...


In [7]:
data = np.loadtxt('./resources/csv/gender/test_gender_img_label_data.csv', delimiter=',', dtype=np.float32)

test_img_data = data[:, 0:-1]
test_label_data = data[:, [-1]]

# min-max scaling
test_img_data = (test_img_data - test_img_data.min()) / (test_img_data.max() - test_img_data.min())
# contrast normalization
test_img_data = (test_img_data - test_img_data.mean()) / test_img_data.std()

In [8]:
learning_rate = 0.0005
keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 80 * 80])
X_img = tf.reshape(X, [-1, 80, 80, 1])
Y = tf.placeholder(tf.float32, [None, 1])

with tf.name_scope('L1') as scope:
    W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
    L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
    L1 = tf.nn.relu6(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
 
with tf.name_scope('L2') as scope:
    # one by one convolution for dimension reduction
    _W2 = tf.Variable(tf.random_normal([1, 1, 32, 5]))
    _L2 = tf.nn.conv2d(L1, _W2, strides=[1, 1, 1, 1], padding='SAME')
    
    W2 = tf.Variable(tf.random_normal([3, 3, 5, 64], stddev=0.01))
    L2 = tf.nn.conv2d(_L2, W2, strides=[1, 1, 1, 1], padding='SAME')
    L2 = tf.nn.relu6(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
with tf.name_scope('L3') as scope:
    # one by one convolution for dimension reduction
    _W3 = tf.Variable(tf.random_normal([1, 1, 64, 5]))
    _L3 = tf.nn.conv2d(L2, _W3, strides=[1, 1, 1, 1], padding='SAME')
    
    W3 = tf.Variable(tf.random_normal([3, 3, 5, 128], stddev=0.01))
    L3 = tf.nn.conv2d(_L3, W3, strides=[1, 1, 1, 1], padding='SAME')
    L3 = tf.nn.relu6(L3)
    L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

with tf.name_scope('L4') as scope:
    # one by one convolution for dimension reduction
    _W4 = tf.Variable(tf.random_normal([1, 1, 128, 5]))
    _L4 = tf.nn.conv2d(L3, _W4, strides=[1, 1, 1, 1], padding='SAME')
    
    W4 = tf.Variable(tf.random_normal([3, 3, 5, 256], stddev=0.01))
    L4 = tf.nn.conv2d(_L4, W4, strides=[1, 1, 1, 1], padding='SAME')
    L4 = tf.nn.relu6(L4)
    L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

with tf.name_scope('L5_FC') as scope:
    L4_flat = tf.reshape(L4, [-1, 5 * 5 * 256])
    W5 = tf.get_variable("W5", shape=[5 * 5 * 256, 5 * 5 * 128], initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([5 * 5 * 128]))
    L5 = tf.nn.relu6(tf.matmul(L4_flat, W5) + b5)
    L5 = tf.nn.dropout(L5, keep_prob=keep_prob)

with tf.name_scope('L6_FC') as scope:
    W6 = tf.get_variable("W6", shape=[5 * 5 * 128, 1], initializer=tf.contrib.layers.xavier_initializer())
    b6 = tf.Variable(tf.random_normal([1]))
    logits = tf.matmul(L5, W6) + b6
    hypothesis = tf.sigmoid(logits)

In [ ]:
# define cost/loss optimizer
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y))
train = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=1e-08).minimize(cost)
prediction = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, Y), dtype=tf.float32))

In [9]:
training_epochs = 120
batch_size = 64
save_model_path = './resources/models/'

tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)
summary = tf.summary.merge_all()

with tf.Session() as sess:
    # for elapsed time
    start_time = time.time()
    
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    
    train_writer = tf.summary.FileWriter('./resources/tensorBoard/gender/train/', sess.graph)
    test_writer = tf.summary.FileWriter('./resources/tensorBoard/gender/test/')
    
    # train my model
    print('Learning started. It takes sometime.')
    
    for epoch in range(training_epochs):
        avg_cost = 0
        avg_acc = 0
        total_batch = int(len(train_img_data) / batch_size)
        
        for i in range(total_batch):
            start = ((i+1) * batch_size) - batch_size
            end = ((i+1) * batch_size)

            batch_xs = train_img_data[start:end]
            batch_ys = train_label_data[start:end]
            feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.5}

            _, c, train_acc, train_s = sess.run([train, cost, accuracy, summary], feed_dict=feed_dict)
            
            avg_cost += c / total_batch
            avg_acc += train_acc / total_batch

        test_acc, test_s = sess.run([accuracy, summary], feed_dict={X: test_img_data, Y: test_label_data, keep_prob: 1})
        
        print('Epoch:', '%d / %d' % (epoch + 1, training_epochs), '  cost: ', '{:.9f}'.format(avg_cost), '  train_Accuracy: %.3f' % (avg_acc * 100), '%', '  test_Accracy: %.3f' % (test_acc * 100), '%')
        
        train_writer.add_summary(train_s, global_step=epoch)
        test_writer.add_summary(test_s, global_step=epoch)

    print('Learning Finished!')
    print('Elapsed Time: ', time.time() - start_time)
    
    saver.save(sess, save_model_path)

Learning started. It takes sometime.
Epoch: 1 / 120   cost:  0.690454098   train_Accuracy: 65.625 %   test_Accracy: 68.000 %
Epoch: 2 / 120   cost:  0.862701305   train_Accuracy: 56.146 %   test_Accracy: 50.000 %
Epoch: 3 / 120   cost:  0.815348876   train_Accuracy: 58.021 %   test_Accracy: 68.000 %
Epoch: 4 / 120   cost:  0.589025825   train_Accuracy: 72.083 %   test_Accracy: 83.000 %
Epoch: 5 / 120   cost:  0.584394175   train_Accuracy: 73.021 %   test_Accracy: 64.000 %
Epoch: 6 / 120   cost:  0.725727340   train_Accuracy: 64.896 %   test_Accracy: 64.000 %
Epoch: 7 / 120   cost:  0.635881128   train_Accuracy: 70.104 %   test_Accracy: 67.000 %
Epoch: 8 / 120   cost:  0.694575413   train_Accuracy: 61.979 %   test_Accracy: 65.000 %
Epoch: 9 / 120   cost:  0.720481724   train_Accuracy: 68.333 %   test_Accracy: 70.000 %
Epoch: 10 / 120   cost:  0.655365318   train_Accuracy: 69.896 %   test_Accracy: 69.000 %
Epoch: 11 / 120   cost:  0.530700558   train_Accuracy: 77.708 %   test_Accracy: 67

Epoch: 93 / 120   cost:  0.000034927   train_Accuracy: 100.000 %   test_Accracy: 97.000 %
Epoch: 94 / 120   cost:  0.000035678   train_Accuracy: 100.000 %   test_Accracy: 97.000 %
Epoch: 95 / 120   cost:  0.000033445   train_Accuracy: 100.000 %   test_Accracy: 97.000 %
Epoch: 96 / 120   cost:  0.000024408   train_Accuracy: 100.000 %   test_Accracy: 98.000 %
Epoch: 97 / 120   cost:  0.000031649   train_Accuracy: 100.000 %   test_Accracy: 98.000 %
Epoch: 98 / 120   cost:  0.000025340   train_Accuracy: 100.000 %   test_Accracy: 97.000 %
Epoch: 99 / 120   cost:  0.000027395   train_Accuracy: 100.000 %   test_Accracy: 98.000 %
Epoch: 100 / 120   cost:  0.000025816   train_Accuracy: 100.000 %   test_Accracy: 98.000 %
Epoch: 101 / 120   cost:  0.000020854   train_Accuracy: 100.000 %   test_Accracy: 98.000 %
Epoch: 102 / 120   cost:  0.000026736   train_Accuracy: 100.000 %   test_Accracy: 98.000 %
Epoch: 103 / 120   cost:  0.000026734   train_Accuracy: 100.000 %   test_Accracy: 97.000 %
Epoch:

In [11]:
save_model_path = './resources/models/'

with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, save_model_path)
    
    a, h, y = sess.run([accuracy, hypothesis, Y], feed_dict={X: test_img_data, Y: test_label_data, keep_prob: 1})
    
    print('Accuracy: ', '{:.2f}'.format(a * 100), '%')
    print('===============================')
    print(sess.run(tf.concat([h, y], 1)))
    

INFO:tensorflow:Restoring parameters from ./resources/models/
Accuracy:  97.00 %
[[9.9914503e-01 1.0000000e+00]
 [9.9999702e-01 1.0000000e+00]
 [4.9971472e-02 0.0000000e+00]
 [1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00]
 [3.5288518e-03 0.0000000e+00]
 [9.7799748e-01 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00]
 [9.9999988e-01 1.0000000e+00]
 [9.9664247e-01 1.0000000e+00]
 [9.9999988e-01 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00]
 [9.9999928e-01 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00]
 [6.9201875e-01 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00]
 [3.9332044e-13 0.0000000e+00]
 [4.4465014e-05 0.0000000e+00]
 [9.9999642e-01 0.0000000e+00]
 [7.9183560e-03 1.0000000e+00]
 [9.9999976e-01 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00]
 [9.9999964e-01 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00]
 [9.9891484e-01 1.0000000e+00]
 [9.9999857e-01 1.0000000e+00]
 [1.1254170e-13 0.00